In [1]:
from IPython.display import HTML

# Cell visibility - COMPLETE:
#tag = HTML('''<style>
#div.input {
#    display:none;
#}
#</style>''')
#display(tag)

#Cell visibility - TOGGLE:
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<p style="text-align:right">
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.</p>''')
display(tag)

## Integrali di polinomi

In questo esempio interattivo è possibile visualizzare alcuni aspetti interessanti riguardanti l'integrale di una funzione polinomiale. Per un dato polinomio (che può essere impostato utilizzando gli slider), l'integrale indefinito verrà calcolato e presentato in modo dinamico, sia nel grafico che in notazione matematica.

Inoltre, impostando il limite inferiore e superiore (utilizzando gli slider dedicati), la rispettiva area sotto la curva verrà evidenziata e calcolata. Si noti che il limite inferiore deve essere inferiore al limite superiore, affinché l'integrale definito sia valido.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import sympy as sym
from IPython.display import Latex, display, clear_output, Markdown # For displaying Markdown and LaTeX code
from ipywidgets import widgets
from ipywidgets import interactive
import matplotlib.patches as mpatches
from scipy.integrate import quad 
from IPython.display import HTML

red_patch = mpatches.Patch(color='red', label='$f(x)$')
blue_patch = mpatches.Patch(color='blue', label='Integrale indefinito di $f(x)$')
gray_patch = mpatches.Patch(color='lightgray', label='Area sotto la curva')
XLIM = 10
YLIM = 30

x = sym.symbols('x')

# Polynomial coeficients
a = 0
b = 0
c = 0
d = 0
e = 0
C = 0


# Sliders
fs_a = widgets.FloatSlider(description='$a$', min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_b = widgets.FloatSlider(description='$b$', min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_c = widgets.FloatSlider(description='$c$',min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_d = widgets.FloatSlider(description='$d$',min=-10.0, max=10.0, step=0.5, continuous_update=False)
fs_e = widgets.FloatSlider(description='$e$',min=-10.0, max=10.0, step=0.5, continuous_update=False)
w_C = widgets.FloatSlider(description='$C$:',min=-10.0, max=10.0, step=0.5, continuous_update=False)
lower_limit = widgets.FloatSlider(description='Limite inf.:',min=-10.0, max=10.0, step=0.5, continuous_update=False)
upper_limit = widgets.FloatSlider(description='Limite sup.:',min=-10.0, max=10.0, step=0.5, continuous_update=False)

# Mathematical notation of a specific (user-defined) polynomial, shown as Markdown
fourth_order = "e + d * x + c * x ** 2 + b * x ** 3 + a * x ** 4"
third_order = "d + c * x + b * x ** 2 + a * x ** 3"
second_order = "c + b * x + a * x ** 2"
first_order = "b + a * x"
zero_order = "a"
tf = sym.sympify(fourth_order)  
w_mark = Markdown('$%s$' %sym.latex(tf))


# General mathematical notation of a polynomial (shown in Label widget)
fourth_order_html = "$f(x)=ax^4$ + $bx^3$ + $cx^2$ + $dx$ + $e$"
third_order_html = "$f(x)=ax^3$ + $bx^2$ + $cx$ + $d$"
second_order_html = "$f(x)=ax^2$ + $bx$ + $c$"
first_order_html = "$f(x)=ax$ + $b$"
zero_order_html = "$f(x)=a$"
w_funLabel = widgets.Label(layout=widgets.Layout(width='40%', margin='0px 0px 0px 50px'),)



dd_order = widgets.Dropdown(
    options=['4', '3', '2', '1', '0'],
    value='4',
    description='Seleziona l\'ordine del polinomio [0-4]:',
    disabled=False,
    style = {'description_width': 'initial'},
)

def dropdown_eventhandler(change):
    fs_a.layout.visibility = 'hidden'
    fs_b.layout.visibility = 'hidden'
    fs_c.layout.visibility = 'hidden'
    fs_d.layout.visibility = 'hidden'
    fs_e.layout.visibility = 'hidden'
    
    if (dd_order.value == '4'):
        fs_a.layout.visibility = 'visible'
        fs_a.description = '$a$'
        fs_b.layout.visibility = 'visible'
        fs_b.description = '$b$'
        fs_c.layout.visibility = 'visible'
        fs_c.description = '$c$'
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$d$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$e$'
        w_funLabel.value=fourth_order_html
    if (dd_order.value == '3'):      
        fs_a.value = 0
        fs_b.layout.visibility = 'visible'
        fs_b.description = '$a$'
        fs_c.layout.visibility = 'visible'
        fs_c.description = '$b$'
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$c$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$d$'
        w_funLabel.value=third_order_html
    if (dd_order.value == '2'):
        fs_a.value = 0
        fs_b.value = 0
        fs_c.layout.visibility = 'visible'
        fs_c.description = '$a$'
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$b$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$c$'
        w_funLabel.value=second_order_html
    if (dd_order.value == '1'):
        fs_a.value = 0
        fs_b.value = 0
        fs_c.value = 0
        fs_d.layout.visibility = 'visible'
        fs_d.description = '$a$'
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$b$'
        w_funLabel.value=first_order_html
    if (dd_order.value == '0'):
        fs_a.value = 0
        fs_b.value = 0
        fs_c.value = 0
        fs_d.value = 0
        fs_e.layout.visibility = 'visible'
        fs_e.description = '$a$'
        w_funLabel.value=zero_order_html

dd_order.observe(dropdown_eventhandler, names='value')



# Functions
def polynomial_function(X_quad, X_cubed, X_squared, X, const, x):
    return const + X * x + X_squared * x ** 2 + X_cubed * x ** 3 + X_quad * x ** 4

def fun(x):
    global a, b, c, d, e
    return e + d * x + c * x ** 2 + b * x ** 3 + a * x ** 4

def f_integral(fx):
    if not fx.is_zero:
        return sym.integrate(fx, x)
    else:
        return ""

def convert(base_text, ss):
    if ss != "":
        tf = sym.sympify(ss)  
        display(Markdown(base_text + '$%s$' %sym.latex(tf)))
         

# Plotting 
def plot_limits(X_quad, X_cubed, X_squared, X, const, ax, a_limit, b_limit):
    ix = np.linspace(a_limit, b_limit)
    iy = polynomial_function(X_quad, X_cubed, X_squared, X, const, ix)
    verts = [(a_limit, 0), *zip(ix, iy), (b_limit, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

    
def plot_function(X_quad, X_cubed, X_squared, X, const, C, llimit, ulimit):
    global a, b, c, d, e, output, x
    a = X_quad
    b = X_cubed
    c = X_squared
    d = X
    e = const
    fig = plt.figure(figsize=(12,6))
    ax = fig.add_subplot(1, 1, 1) 
    
    # Plot input function
    x_p = np.linspace(-XLIM, XLIM, num=1000)
    y_p = polynomial_function(X_quad, X_cubed, X_squared, X, const, x_p)
    plt.plot(x_p, y_p, 'r-')

    # Plot indefinite integral of the input function
    integ = f_integral(fun(x))
    #integ = integ + str(C)
    if integ != "":
        if C < 0:
            integ = str(integ) + "-" + str(abs(C))
        else:
            integ = str(integ)+ "+" + str(C)
        f_integrate = sym.lambdify(x, integ) # from str to function
        x_p = np.linspace(-XLIM, XLIM, num=1000)
        y_p = f_integrate(x_p)
        ax.plot(x_p, y_p, 'b-', linewidth=2)

    # Plot integral limits (area under curve)
    if ulimit < llimit:
        display(Markdown('Il limite superiore deve essere maggiore o uguale del limite inferiore'))
        res = "";
    else:
        plot_limits(X_quad, X_cubed, X_squared, X, const, ax, llimit, ulimit)
        res, err = quad(fun, llimit, ulimit)
    
    plt.grid(True)
    plt.xlim(-XLIM, XLIM)
    plt.ylim(-YLIM, YLIM)
    plt.axhline(y=0,lw=0.8,color='k')
    plt.axvline(x=0,lw=0.8,color='k')
    plt.xlabel('x')
    plt.ylabel('$f(x)$, integrale indefinito di $f(x)$')
    plt.legend(handles=[red_patch, blue_patch, gray_patch])
    plt.show()

    convert("$f(x)$: ", fun(x))
    if integ != "":
        if C == 0:
            integ_str = str(integ) + "+ C" 
        else:
            integ_str = str(integ)
        convert("Integrale indefinito di $f(x)$: ", integ_str)
    if res != "":
        display(Markdown('Area sotto la curva: ' + str(res)))

w_funLabel.value=fourth_order_html
control_widgets = widgets.HBox()
control_widgets.children=[dd_order, w_funLabel]
display(control_widgets)
interactive(plot_function, const=fs_e, X=fs_d, X_squared=fs_c, X_cubed=fs_b, X_quad = fs_a, C = w_C, llimit=lower_limit, ulimit=upper_limit)


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='$a$', max=10.0, min=-10.0, …